# Convert the json to networkx graphs format

In [22]:
import sys
# append parent directory to path
sys.path.append("..")
from datasets.VG_graphs import get_realistic_graphs_dataset
dataset = get_realistic_graphs_dataset('v1', 'attr')

100


# Evaluate

In [23]:
from jt_training import get_dataloader, train_one_epoch, evaluate, get_free_gpu

mode = "text_embeddings"

def load_model(clip_model_type, clip_pretrained_dataset, n_rel_classes, n_obj_classes, n_attr_classes, shallow=True, input_mode="text_embeddings", obj_heads=False):
    from open_clip.jt_ViT_RelClassifier_lightning import ViT_RelClassifier
    model = ViT_RelClassifier(n_rel_classes, n_obj_classes, n_attr_classes, clip_model_type, clip_pretrained_dataset, shallow=shallow, mode=input_mode, with_object_heads=obj_heads)
    prepocess_function = model.preprocess
    device = get_free_gpu(min_mem=20000)
    print(f"Using device {device}")
    model.to(device)
    return model, prepocess_function, device

In [32]:
import torch
clip_model_type = 'ViT-L-14' # 'ViT-L-14' #'ViT-B/32'
clip_pretrained_dataset = 'laion2b_s32b_b82k' # 'laion2b_s32b_b82k' #'laion400m_e32'
image_dir = "/local/home/jthomm/GraphCLIP/datasets/visual_genome/raw/VG/"
metadata_path = "/local/home/jthomm/GraphCLIP/datasets/visual_genome/processed/"

model, prepocess_function, device = load_model(clip_model_type, clip_pretrained_dataset, 100, 200, 100, obj_heads=True)
# loaded = torch.load("/local/home/jthomm/GraphCLIP/experiments/2023-05-05/vision_transformer_12/best_rel_model.ckpt")
# loaded = torch.load("/local/home/jthomm/GraphCLIP/experiments/2023-05-09/vision_transformer_5/best_rel_model.ckpt", map_location=device)
# new two ViT/B32 models:
# no attribute wighting
# loaded = torch.load("/local/home/jthomm/GraphCLIP/experiments/2023-05-22/vision_transformer_8/best_rel_model.ckpt", map_location=device)
# attribute wighting
# loaded = torch.load("/local/home/jthomm/GraphCLIP/experiments/2023-05-22/vision_transformer_7/best_rel_model.ckpt", map_location=device)

# new models with different learning rates
# loaded = torch.load("/local/home/jthomm/GraphCLIP/experiments/2023-05-23/vision_transformer_4/best_rel_model.ckpt", map_location=device)
# loaded = torch.load("/local/home/jthomm/GraphCLIP/experiments/2023-05-23/vision_transformer_5/best_rel_model.ckpt", map_location=device)

# loaded = torch.load("/local/home/jthomm/GraphCLIP/experiments/2023-05-24/vision_transformer_0/best_rel_model.ckpt", map_location=device)

# Big ViT-L
# loaded = torch.load('/local/home/jthomm/GraphCLIP/experiments/2023-05-25/vision_transformer_6/best_rel_model.ckpt', map_location=torch.device('cpu'))
loaded = torch.load('/local/home/jthomm/GraphCLIP/experiments/2023-05-27/vision_transformer_39/model_epoch-v8.ckpt', map_location=torch.device('cpu'))
print(loaded.keys())
model.load_state_dict(loaded['state_dict'])


Using text embeddings as input to the model.
Using device cuda:7
dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters', 'datamodule_hparams_name', 'datamodule_hyper_parameters'])


<All keys matched successfully>

In [33]:
FILTERED_OBJECTS = ['man', 'person', 'window', 'tree', 'building', 'shirt', 'wall', 'woman', 'sign', 'sky', 'ground', 'grass', 'table', 'pole', 'head', 'light', 'water', 'car', 'hand', 'hair', 'people', 'leg', 'trees', 'clouds', 'ear', 'plate', 'leaves', 'fence', 'door', 'pants', 'eye', 'train', 'chair', 'floor', 'road', 'street', 'hat', 'snow', 'wheel', 'shadow', 'jacket', 'nose', 'boy', 'line', 'shoe', 'clock', 'sidewalk', 'boat', 'tail', 'cloud', 'handle', 'letter', 'girl', 'leaf', 'horse', 'bus', 'helmet', 'bird', 'giraffe', 'field', 'plane', 'flower', 'elephant', 'umbrella', 'dog', 'shorts', 'arm', 'zebra', 'face', 'windows', 'sheep', 'glass', 'bag', 'cow', 'bench', 'cat', 'food', 'bottle', 'rock', 'tile', 'kite', 'tire', 'post', 'number', 'stripe', 'surfboard', 'truck', 'logo', 'glasses', 'roof', 'skateboard', 'motorcycle', 'picture', 'flowers', 'bear', 'player', 'foot', 'bowl', 'mirror', 'background', 'pizza', 'bike', 'shoes', 'spot', 'tracks', 'pillow', 'shelf', 'cap', 'mouth', 'box', 'jeans', 'dirt', 'lights', 'legs', 'house', 'part', 'trunk', 'banana', 'top', 'plant', 'cup', 'counter', 'board', 'bed', 'wave', 'bush', 'ball', 'sink', 'button', 'lamp', 'beach', 'brick', 'flag', 'neck', 'sand', 'vase', 'writing', 'wing', 'paper', 'seat', 'lines', 'reflection', 'coat', 'child', 'toilet', 'laptop', 'airplane', 'letters', 'glove', 'vehicle', 'phone', 'book', 'branch', 'sunglasses', 'edge', 'cake', 'desk', 'rocks', 'frisbee', 'tie', 'tower', 'animal', 'hill', 'mountain', 'headlight', 'ceiling', 'cabinet', 'eyes', 'stripes', 'wheels', 'lady', 'ocean', 'racket', 'container', 'skier', 'keyboard', 'towel', 'frame', 'windshield', 'hands', 'back', 'track', 'bat', 'finger', 'pot', 'orange', 'fork', 'waves', 'design', 'feet', 'basket', 'fruit', 'broccoli', 'engine', 'guy', 'knife', 'couch', 'railing', 'collar', 'cars']
FILTERED_RELATIONSHIPS = ['on', 'has', 'in', 'of', 'wearing', 'with', 'behind', 'holding', 'on a', 'near', 'on top of', 'next to', 'has a', 'under', 'of a', 'by', 'above', 'wears', 'in front of', 'sitting on', 'on side of', 'attached to', 'wearing a', 'in a', 'over', 'are on', 'at', 'for', 'around', 'beside', 'standing on', 'riding', 'standing in', 'inside', 'have', 'hanging on', 'walking on', 'on front of', 'are in', 'hanging from', 'carrying', 'holds', 'covering', 'belonging to', 'between', 'along', 'eating', 'and', 'sitting in', 'watching', 'below', 'painted on', 'laying on', 'against', 'playing', 'from', 'inside of', 'looking at', 'with a', 'parked on', 'to', 'has an', 'made of', 'covered in', 'mounted on', 'says', 'growing on', 'across', 'part of', 'on back of', 'flying in', 'outside', 'lying on', 'worn by', 'walking in', 'sitting at', 'printed on', 'underneath', 'crossing', 'beneath', 'full of', 'using', 'filled with', 'hanging in', 'covered with', 'built into', 'standing next to', 'adorning', 'a', 'in middle of', 'flying', 'supporting', 'touching', 'next', 'swinging', 'pulling', 'growing in', 'sitting on top of', 'standing', 'lying on top of']
FILTERED_ATTRIBUTES = ['white', 'black', 'blue', 'green', 'red', 'brown', 'yellow', 'small', 'large', 'wooden', 'gray', 'silver', 'metal', 'orange', 'grey', 'tall', 'long', 'dark', 'pink', 'clear', 'standing', 'round', 'tan', 'glass', 'here', 'wood', 'open', 'purple', 'big', 'short', 'plastic', 'parked', 'sitting', 'walking', 'striped', 'brick', 'young', 'gold', 'old', 'hanging', 'empty', 'on', 'bright', 'concrete', 'cloudy', 'colorful', 'one', 'beige', 'bare', 'wet', 'light', 'square', 'little', 'closed', 'stone', 'blonde', 'shiny', 'thin', 'dirty', 'flying', 'smiling', 'painted', 'thick', 'part', 'sliced', 'playing', 'tennis', 'calm', 'leather', 'distant', 'rectangular', 'looking', 'grassy', 'dry', 'light brown', 'cement', 'leafy', 'wearing', 'tiled', "man's", 'light blue', 'baseball', 'cooked', 'pictured', 'curved', 'decorative', 'dead', 'eating', 'paper', 'paved', 'fluffy', 'lit', 'back', 'framed', 'plaid', 'dirt', 'watching', 'colored', 'stuffed', 'circular']
rel_classes = {rel:i for i,rel in enumerate(FILTERED_RELATIONSHIPS)}
obj_classes = {obj:i for i,obj in enumerate(FILTERED_OBJECTS)}
attr_classes = {attr:i for i,attr in enumerate(FILTERED_ATTRIBUTES)}
import os
obj_embeddings = torch.load(os.path.join('/local/home/jthomm/GraphCLIP/datasets/visual_genome/processed', 'filtered_object_label_embeddings.pt'), map_location=device)
text_embeddings = {obj:torch.tensor(obj_embeddings[i]) for i,obj in enumerate(FILTERED_OBJECTS)}

In [43]:
import random
import PIL
from tqdm import tqdm
# dataset_orig, dataset_adv, list_to_iterate = get_realistic_graphs_dataset_ViT(prepocess_function, image_dir, mode=mode, version='v2')
correct = 0
total = 0
orig_confidences = []
adv_confidences = []
for d in tqdm(dataset):
    original_graph = d['original_graph']
    adv_graph = d['adv_graph']
    image_id = original_graph.image_id

    # load image
    image = PIL.Image.open(image_dir + str(image_id) + ".jpg").convert("RGB")
    image = prepocess_function(image).unsqueeze(0).to(device)

    g_attr_confidences = []
    def get_attr_confidence(g):
        for node in g.nodes:
            text_embd_obj = text_embeddings[g.nodes[node]['name']].to(device)
            full_text_clip_embd = torch.cat((text_embd_obj, text_embd_obj), dim=0).reshape(1,-1)
            obj_label = torch.tensor(obj_classes[g.nodes[node]['name']])
            rel, obj1, obj2, attr1, attr2 = model(image, full_text_clip_embd.unsqueeze(0))
            attr = torch.sigmoid((attr1+attr2)/2)
            attr_labels = [attr_classes[attr] for attr in g.nodes[node]['attributes']]
            for attr_label in attr_labels:
                g_attr_confidences.append(attr[0][attr_label].item())
            # text_embd_obj = text_embeddings[g.nodes[node]['name']].to(device)
            # full_text_clip_embd = torch.cat((text_embd_obj, text_embd_obj), dim=0).reshape(1,-1)
            # obj_label = torch.tensor(obj_classes[g.nodes[node]['name']])
            # rel, obj1, obj2, attr1, attr2 = model(image.unsqueeze(0), full_text_clip_embd.unsqueeze(0))
            # # mean of attr1 and attr2
            # attr = (attr1 + attr2) / 2
            # attr = torch.sigmoid(attr)
            # attr_labels = [attr_classes[attr] for attr in g.nodes[node]['attributes']]
            # for attr_label in attr_labels:
            #     g_attr_confidences.append(attr[0][attr_label].item())
        if len(g_attr_confidences) > 0:
            attr_confidence = torch.mean(torch.tensor(g_attr_confidences)).cpu().item()
        else:
            assert False, "no attributes found"
        return attr_confidence
    
    original_attr_confidence = get_attr_confidence(original_graph)
    orig_confidences.append(original_attr_confidence)
    adv_attr_confidence = get_attr_confidence(adv_graph)
    adv_confidences.append(adv_attr_confidence)

    if original_attr_confidence >= adv_attr_confidence:
        correct += 1
    else:
        pass
        # rel_name_dict = dataset_orig.rel_classes # a dict with the relationship names as keys and the ids as values
        # rel_name = [key for key, value in rel_name_dict.items() if value == rel_label][0]
        # rel_name_adv = [key for key, value in rel_name_dict.items() if value == rel_label_adv][0]
        # print("orig rel:", rel_name, "adv rel:", rel_name_adv, "orig conf:", original_rel_confidence, "adv conf:", adversarial_rel_confidence)
    total += 1
    # print("original relationship confidence:", original_rel_confidence)
    # print("adversarial relationship confidence:", adversarial_rel_confidence)
    # break
print("accuracy:", correct/total)
print(orig_confidences)
print(adv_confidences)

100%|██████████| 100/100 [00:08<00:00, 11.29it/s]

accuracy: 0.9
[0.5067164301872253, 0.5126970410346985, 0.014940398745238781, 0.053423136472702026, 0.35332536697387695, 0.532669186592102, 0.5962006449699402, 0.43216270208358765, 0.8438829779624939, 0.1987074315547943, 0.40836283564567566, 0.30747613310813904, 0.014071589335799217, 0.30224454402923584, 0.3189968764781952, 0.870049774646759, 0.4589889645576477, 0.48020416498184204, 0.4660971760749817, 0.39960652589797974, 0.8092999458312988, 0.1367560774087906, 0.9554212689399719, 0.3491665720939636, 0.18109504878520966, 0.473165363073349, 0.26860034465789795, 0.4498293399810791, 0.2727140188217163, 0.15753072500228882, 0.45829343795776367, 0.037026457488536835, 0.2376910150051117, 0.26526564359664917, 0.09565319865942001, 0.2229175865650177, 0.010786309838294983, 0.8500232696533203, 0.0007952185114845634, 0.8031555414199829, 0.6283318996429443, 0.4280907213687897, 0.5244888067245483, 0.8952549695968628, 0.6321170330047607, 0.3447175920009613, 0.25000450015068054, 0.3405395746231079, 0

In [42]:
print("accuracy:", correct/total, "total:", total, "correct:", correct)
print(orig_confidences)
print(adv_confidences)

accuracy: 0.9 total: 100 correct: 90
[0.5067164301872253, 0.5126970410346985, 0.014940381050109863, 0.05342324823141098, 0.3533254861831665, 0.5326694250106812, 0.5962003469467163, 0.43216264247894287, 0.8438830375671387, 0.19870735704898834, 0.40836259722709656, 0.30747610330581665, 0.01407160609960556, 0.30224448442459106, 0.3189970552921295, 0.8700499534606934, 0.4589890241622925, 0.4802042245864868, 0.4660971760749817, 0.39960646629333496, 0.8092999458312988, 0.13675600290298462, 0.9554212689399719, 0.3491666913032532, 0.18109509348869324, 0.4731655716896057, 0.268600195646286, 0.4498292803764343, 0.272714227437973, 0.15753062069416046, 0.45829349756240845, 0.037026479840278625, 0.2376909703016281, 0.2652658224105835, 0.09565316140651703, 0.22291754186153412, 0.010786313563585281, 0.8500231504440308, 0.0007952179294079542, 0.8031555414199829, 0.6283320188522339, 0.42809051275253296, 0.5244889855384827, 0.8952550292015076, 0.632116973400116, 0.34471777081489563, 0.2500043213367462, 